In [21]:
%pip install pypdf sentence-transformers replicate

   ---------------------------------------- 0.0/75.9 kB ? eta -:--:--
   ---------------------------------------- 75.9/75.9 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/77.0 kB ? eta -:--:--
   ---------------------------------------- 77.0/77.0 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, HypotheticalDocumentEmbedder, HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.llms import OpenAI, Replicate
from langchain.chains import VectorDBQA, ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from dotenv import load_dotenv
import os, sys

load_dotenv()

True

In [8]:
root_folder = 'H:/Meu Drive/Prog/IA/Daily Dose of Data Science - Archive.pdf'
persist_directory = './data/processed'
# Load and process the text
if not(os.path.exists(persist_directory)):
    loader = PyPDFLoader(root_folder)
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    # Embed and store the texts
    embedding = HuggingFaceEmbeddings()
    vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)
    vectordb.persist()
else:
    # Now we can load the persisted database from disk, and use it as normal. 
    embedding = HuggingFaceEmbeddings()
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

d:\# Tropical Brain Innovation\llm-deploy-locally\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# Initialize Replicate Llama2 Model https://medium.com/@woyera/how-to-chat-with-your-pdf-using-python-llama-2-41df80c4e674
llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    model_kwargs={"temperature": 0.75, "max_length": 3000}
)

In [10]:
# Set up the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectordb.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True
)

In [11]:
# Start chatting with the chatbot
chat_history = []
while True:
    query = input('Prompt: ')
    if query.lower() in ["exit", "quit", "q"]:
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))

d:\# Tropical Brain Innovation\llm-deploy-locally\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Answer: Normalization is useful when the data is not on the same scale, to prevent features from dominating the model's output, and to improve the model's robustness to variations in the data. It is not always necessary, but it is important to know when to do it. The following visual helps to determine which algorithms typically need normalized data and which don't.

Answer: Normalization is necessary in scenarios where the data is not uniformly distributed or where the features have different scales. For example, in linear regression, normalization of the input features is necessary to prevent the model from being biased towards a particular feature. Similarly, in logistic regression, normalization of the input features is necessary to prevent the model from overfitting or underfitting. Additionally, normalization is also necessary in scenarios where the data is imbalanced, meaning that one class has more instances than the others. This is because normalization helps to ensure that al

SystemExit: 

d:\# Tropical Brain Innovation\llm-deploy-locally\.venv\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
query='when we use normalization for machine learning models?'
llm.run(query)

AttributeError: 'Replicate' object has no attribute 'run'

In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()